# Fetch Codebase

In [1]:
#@title
import os
#os.chdir('/content')
#CODE_DIR = 'sefa'
#!git clone https://github.com/genforce/sefa.git $CODE_DIR
#os.chdir(f'./{CODE_DIR}')
os.chdir("./sefa-cryowgan")
os.getcwd()

'/home/zami/projects/Factorized_CryoEM_wgan/sefa-cryowgan'

In [2]:
os.listdir()

['SessionState.py',
 'LICENSE',
 'README.md',
 'interface.py',
 'sefa.py',
 'latent_codes',
 '__pycache__',
 '.ipynb_checkpoints',
 'models',
 'checkpoints',
 'utils.py']

# Define Utility Functions

In [3]:
from models import MODEL_ZOO
print(MODEL_ZOO.keys())

dict_keys(['cryoEM_wgan', 'pggan_celebahq1024', 'pggan_bedroom256', 'pggan_livingroom256', 'pggan_diningroom256', 'pggan_kitchen256', 'pggan_church256', 'pggan_tower256', 'pggan_bridge256', 'pggan_restaurant256', 'pggan_classroom256', 'pggan_conferenceroom256', 'pggan_person256', 'pggan_cat256', 'pggan_dog256', 'pggan_bird256', 'pggan_horse256', 'pggan_sheep256', 'pggan_cow256', 'pggan_car256', 'pggan_bicycle256', 'pggan_motorbike256', 'pggan_bus256', 'pggan_train256', 'pggan_boat256', 'pggan_airplane256', 'pggan_bottle256', 'pggan_chair256', 'pggan_pottedplant256', 'pggan_tvmonitor256', 'pggan_diningtable256', 'pggan_sofa256', 'stylegan_ffhq1024', 'stylegan_celebahq1024', 'stylegan_bedroom256', 'stylegan_cat256', 'stylegan_car512', 'stylegan_celeba_partial256', 'stylegan_ffhq256', 'stylegan_ffhq512', 'stylegan_livingroom256', 'stylegan_diningroom256', 'stylegan_kitchen256', 'stylegan_apartment256', 'stylegan_church256', 'stylegan_tower256', 'stylegan_bridge256', 'stylegan_restaurant25

In [4]:
#@title
import os.path
import io
import IPython.display
import numpy as np
import cv2
import PIL.Image

import torch

from models import parse_gan_type
from utils import to_tensor
from utils import postprocess
from utils import load_generator
from utils import factorize_weight


def sample(generator, gan_type, num=1, seed=11):
    """Samples latent codes."""
    #torch.manual_seed(seed)                                                                    TURNED OFF SEEDING!!!!
    #codes = torch.randn(num, generator.z_space_dim).cuda()     CHANGED FOR OUR GAN
    codes = torch.randn((num, 100, 1, 1)).cuda()
    codes = pixel_norm_layer(generator.net[0](codes))
    '''
    if gan_type == 'pggan':
        codes = generator.layer0.pixel_norm(codes)
    elif gan_type == 'stylegan':
        codes = generator.mapping(codes)['w']
        codes = generator.truncation(codes, trunc_psi=0.7, trunc_layers=8)
    elif gan_type == 'stylegan2':
        codes = generator.mapping(codes)['w']
        codes = generator.truncation(codes, trunc_psi=0.5, trunc_layers=18)
    '''
    codes = codes.detach().cpu().numpy()
    return codes

def pixel_norm_layer(x):
    """Implements pixel-wise feature vector normalization layer."""
    norm = torch.sqrt(torch.mean(x ** 2, dim=1, keepdim=True) + 1e-8)
    return x / norm


def synthesize(generator, gan_type, codes):
    """Synthesizes images with the give codes."""
    '''
  if gan_type == 'pggan':
    images = generator(to_tensor(codes))['image']
  elif gan_type == 'cryowgan':   #################################FOR US!!
     return generator(to_tensor(codes))
  elif gan_type in ['stylegan', 'stylegan2']:
    images = generator.synthesis(to_tensor(codes))['image']
      '''
    #images = generator(to_tensor(codes)) ##########
    images = generator(to_tensor(codes), synthesize=False)
    images = postprocess(images)
    return images


from PIL import Image


def imshow(images, col, viz_size=128):
    """Shows images in one figure."""
    num, height, width, channels = images.shape
    assert num % col == 0
    row = num // col

    # Create an empty array to hold the fused image
    # Initialize the fused_image with 3 channels (for RGB)
    fused_image = np.zeros((viz_size * row, viz_size * col, 3), dtype=np.uint8)

    for idx, image in enumerate(images):
        i, j = divmod(idx, col)
        y = i * viz_size
        x = j * viz_size
        
        # Resize image if it's not the desired size
        if height != viz_size or width != viz_size:
            image = cv2.resize(image, (viz_size, viz_size))
        
        # Handle grayscale images by converting them to 3 channels
        if channels == 1:
            image = np.repeat(image, 3, axis=-1)  # Repeat the grayscale image across 3 channels
        
        # Place the image into the correct position in the fused image
        fused_image[y:y + viz_size, x:x + viz_size] = image

    # Convert to uint8 if not already in that format
    fused_image = np.asarray(fused_image, dtype=np.uint8)

    # Directly display the image in the notebook
    pil_image = Image.fromarray(fused_image)
    pil_image.show()

    return pil_image


# Select a Model

In [5]:
#@title { display-mode: "form", run: "auto" }
model_name = "cryoEM_wgan" #@param ['stylegan_animeface512', 'stylegan_car512', 'stylegan_cat256', 'pggan_celebahq1024', 'stylegan_bedroom256']

generator = load_generator(model_name, '../sefa-cryowgan/checkpoints/run26_10k_checkpoint_netG.tar')
gan_type = parse_gan_type(generator)

Building generator for model `cryoEM_wgan` ...
Finish building generator.
Loading checkpoint from `../sefa-cryowgan/checkpoints/run26_10k_checkpoint_netG.tar` ...
Finish loading checkpoint.


# Sample Latent Codes

In [6]:
#@title { display-mode: "form", run: "auto" }

num_samples = 12 #@param {type:"slider", min:1, max:8, step:1}
noise_seed = 0 #@param {type:"slider", min:0, max:1000, step:1}

codes = torch.randn((num_samples,100,1,1)).detach().numpy()
print(codes.shape)

images2 = synthesize(generator, gan_type, codes)#codes) #torch.randn((num_samples, 100, 1, 1)).cuda()
print(images2.shape)
#imshow(images2, col=num_samples)

(12, 100, 1, 1)
(12, 128, 128, 1)


In [18]:
print(imshow(images2, col=3))

<PIL.Image.Image image mode=RGB size=384x512 at 0x7F19F97C7700>


# Factorize & Edit

In [13]:
#@title { display-mode: "form", run: "auto" }

layer_idx = "0" #@param ['all', '0-1', '2-5', '6-13']
semantic_1 = -1 #@param {type:"slider", min:-3.0, max:3.0, step:0.1}
semantic_2 = 0 #@param {type:"slider", min:-3.0, max:3.0, step:0.1}
semantic_3 = 0 #@param {type:"slider", min:-3.0, max:3.0, step:0.1}
semantic_4 = 0 #@param {type:"slider", min:-3.0, max:3.0, step:0.1}
semantic_5 = 0 #@param {type:"slider", min:-3.0, max:3.0, step:0.1}

# Fast implementation to factorize the weight by SeFa.
layers, boundaries, _ = factorize_weight(generator, layer_idx)

new_codes = codes.copy()
for sem_idx in range(5):
  boundary = boundaries[sem_idx:sem_idx + 1]
  step = eval(f'semantic_{sem_idx + 1}')
  if gan_type == 'cryowgan':
    new_codes += (boundary.reshape(1, 100, 1, 1) * step)  #CHANGED TO INCLUDE CRYOWGAN
    
  elif gan_type =='pggan':            
    new_codes += boundary * step
    
  elif gan_type in ['stylegan', 'stylegan2']:
    new_codes[:, layers, :] += boundary * step
new_images = synthesize(generator, gan_type, new_codes)
print(imshow(new_images, col=3))

<PIL.Image.Image image mode=RGB size=384x512 at 0x7F199C54BF70>


In [17]:
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import clear_output

# Define your layer and semantic controls as interactive widgets
layer_idx = widgets.Dropdown(
    options=['all', '0-1', '2-5', '6-13'],
    value='0-1',
    description='Layer Index:',
)

semantic_1 = widgets.FloatSlider(
    value=0,
    min=-3.0,
    max=3.0,
    step=0.1,
    description='Semantic 1:',
    continuous_update=True,
)

semantic_2 = widgets.FloatSlider(
    value=0,
    min=-3.0,
    max=3.0,
    step=0.1,
    description='Semantic 2:',
    continuous_update=True,
)

semantic_3 = widgets.FloatSlider(
    value=0,
    min=-3.0,
    max=3.0,
    step=0.1,
    description='Semantic 3:',
    continuous_update=True,
)

semantic_4 = widgets.FloatSlider(
    value=0,
    min=-3.0,
    max=3.0,
    step=0.1,
    description='Semantic 4:',
    continuous_update=True,
)

semantic_5 = widgets.FloatSlider(
    value=0,
    min=-3.0,
    max=3.0,
    step=0.1,
    description='Semantic 5:',
    continuous_update=True,
)

# Define a function that will update the visuals in real-time
def update_visual(layer_idx, semantic_1, semantic_2, semantic_3, semantic_4, semantic_5):
    clear_output(wait=True)  # Clear previous output to update in real time

    '''
    # Display the selected values
    print(f'Layer Index: {layer_idx}')
    print(f'Semantic 1: {semantic_1}')
    print(f'Semantic 2: {semantic_2}')
    print(f'Semantic 3: {semantic_3}')
    print(f'Semantic 4: {semantic_4}')
    print(f'Semantic 5: {semantic_5}')
    '''
    # Fast implementation to factorize the weight by SeFa.
    layers, boundaries, _ = factorize_weight(generator, layer_idx)

    new_codes = codes.copy()
    for sem_idx, step in enumerate([semantic_1, semantic_2, semantic_3, semantic_4, semantic_5]):
        boundary = boundaries[sem_idx:sem_idx + 1]
        if gan_type == 'pggan':
            new_codes += boundary * step
        elif gan_type == 'cryowgan':
            new_codes += (boundary.reshape(1, 100, 1, 1) * step)  #CHANGED TO INCLUDE CRYOWGAN
        elif gan_type in ['stylegan', 'stylegan2']:
            new_codes[:, layers, :] += boundary * step
    new_images = synthesize(generator, gan_type, new_codes)
    imshow(new_images, col=3)

# Use `interact` to create the interactive UI and trigger `update_visual`
interact(update_visual, 
         layer_idx=layer_idx, 
         semantic_1=semantic_1, 
         semantic_2=semantic_2, 
         semantic_3=semantic_3, 
         semantic_4=semantic_4, 
         semantic_5=semantic_5)


interactive(children=(Dropdown(description='Layer Index:', index=1, options=('all', '0-1', '2-5', '6-13'), val…

<function __main__.update_visual(layer_idx, semantic_1, semantic_2, semantic_3, semantic_4, semantic_5)>